In [1]:
from __future__ import print_function, division

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=10000, batch_size=32, save_interval=50)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 8, 8, 64)          0

67 [D loss: 0.810332, acc.: 53.12%] [G loss: 0.846990]
68 [D loss: 0.753712, acc.: 59.38%] [G loss: 1.052601]
69 [D loss: 0.724252, acc.: 62.50%] [G loss: 1.233110]
70 [D loss: 0.849578, acc.: 50.00%] [G loss: 1.118565]
71 [D loss: 0.819969, acc.: 51.56%] [G loss: 1.391551]
72 [D loss: 0.841670, acc.: 54.69%] [G loss: 1.461295]
73 [D loss: 0.924768, acc.: 48.44%] [G loss: 1.107222]
74 [D loss: 0.791738, acc.: 54.69%] [G loss: 1.130447]
75 [D loss: 0.768781, acc.: 56.25%] [G loss: 1.302967]
76 [D loss: 0.719166, acc.: 56.25%] [G loss: 1.196037]
77 [D loss: 0.695258, acc.: 59.38%] [G loss: 1.211389]
78 [D loss: 0.680041, acc.: 57.81%] [G loss: 1.004768]
79 [D loss: 0.776173, acc.: 48.44%] [G loss: 1.107984]
80 [D loss: 0.906997, acc.: 48.44%] [G loss: 0.963475]
81 [D loss: 0.776083, acc.: 56.25%] [G loss: 1.149548]
82 [D loss: 0.798076, acc.: 48.44%] [G loss: 1.169410]
83 [D loss: 0.747126, acc.: 59.38%] [G loss: 1.144437]
84 [D loss: 0.728521, acc.: 57.81%] [G loss: 1.134328]
85 [D loss

217 [D loss: 0.738965, acc.: 48.44%] [G loss: 1.051343]
218 [D loss: 0.643001, acc.: 62.50%] [G loss: 1.361726]
219 [D loss: 0.695496, acc.: 57.81%] [G loss: 1.161921]
220 [D loss: 0.939337, acc.: 42.19%] [G loss: 0.805906]
221 [D loss: 0.869809, acc.: 46.88%] [G loss: 0.900154]
222 [D loss: 0.719072, acc.: 53.12%] [G loss: 1.115194]
223 [D loss: 0.729036, acc.: 60.94%] [G loss: 1.009493]
224 [D loss: 0.768050, acc.: 56.25%] [G loss: 1.123372]
225 [D loss: 0.820608, acc.: 46.88%] [G loss: 1.104354]
226 [D loss: 0.767624, acc.: 46.88%] [G loss: 1.083774]
227 [D loss: 0.654459, acc.: 62.50%] [G loss: 1.190993]
228 [D loss: 0.692616, acc.: 62.50%] [G loss: 1.060532]
229 [D loss: 0.766874, acc.: 53.12%] [G loss: 1.216064]
230 [D loss: 0.550835, acc.: 67.19%] [G loss: 1.063857]
231 [D loss: 0.727534, acc.: 67.19%] [G loss: 1.168947]
232 [D loss: 0.600189, acc.: 70.31%] [G loss: 1.390911]
233 [D loss: 0.550200, acc.: 76.56%] [G loss: 1.028914]
234 [D loss: 0.672652, acc.: 64.06%] [G loss: 1.

367 [D loss: 1.021316, acc.: 42.19%] [G loss: 0.996845]
368 [D loss: 0.737900, acc.: 51.56%] [G loss: 1.081146]
369 [D loss: 0.431618, acc.: 79.69%] [G loss: 1.234956]
370 [D loss: 0.656552, acc.: 62.50%] [G loss: 1.058400]
371 [D loss: 0.332384, acc.: 89.06%] [G loss: 0.953061]
372 [D loss: 0.269001, acc.: 90.62%] [G loss: 1.079246]
373 [D loss: 0.158477, acc.: 98.44%] [G loss: 1.116871]
374 [D loss: 0.193566, acc.: 93.75%] [G loss: 0.872031]
375 [D loss: 0.273686, acc.: 92.19%] [G loss: 1.031543]
376 [D loss: 0.308895, acc.: 85.94%] [G loss: 1.164325]
377 [D loss: 0.969701, acc.: 53.12%] [G loss: 0.862298]
378 [D loss: 0.698985, acc.: 62.50%] [G loss: 1.173135]
379 [D loss: 0.381466, acc.: 82.81%] [G loss: 1.157585]
380 [D loss: 0.661270, acc.: 65.62%] [G loss: 1.220971]
381 [D loss: 0.357485, acc.: 85.94%] [G loss: 1.392286]
382 [D loss: 0.841622, acc.: 51.56%] [G loss: 0.863442]
383 [D loss: 0.280366, acc.: 89.06%] [G loss: 1.203308]
384 [D loss: 0.280267, acc.: 93.75%] [G loss: 1.

517 [D loss: 0.281998, acc.: 90.62%] [G loss: 1.394189]
518 [D loss: 0.330866, acc.: 90.62%] [G loss: 1.282002]
519 [D loss: 0.270616, acc.: 90.62%] [G loss: 1.176231]
520 [D loss: 0.094322, acc.: 98.44%] [G loss: 1.065666]
521 [D loss: 0.331834, acc.: 85.94%] [G loss: 1.188325]
522 [D loss: 0.347311, acc.: 82.81%] [G loss: 0.817509]
523 [D loss: 0.013385, acc.: 100.00%] [G loss: 1.325314]
524 [D loss: 0.026421, acc.: 100.00%] [G loss: 1.547040]
525 [D loss: 0.116311, acc.: 100.00%] [G loss: 1.150499]
526 [D loss: 0.016911, acc.: 100.00%] [G loss: 1.206651]
527 [D loss: 0.093403, acc.: 98.44%] [G loss: 1.503858]
528 [D loss: 0.045852, acc.: 100.00%] [G loss: 1.062728]
529 [D loss: 0.040145, acc.: 100.00%] [G loss: 0.961361]
530 [D loss: 0.028714, acc.: 100.00%] [G loss: 1.059532]
531 [D loss: 0.071210, acc.: 100.00%] [G loss: 1.185032]
532 [D loss: 0.336411, acc.: 84.38%] [G loss: 0.801110]
533 [D loss: 0.165955, acc.: 92.19%] [G loss: 1.237279]
534 [D loss: 0.274156, acc.: 92.19%] [G 

...

9359 [D loss: 0.000279, acc.: 100.00%] [G loss: 0.024378]
9360 [D loss: 0.000817, acc.: 100.00%] [G loss: 0.036070]
9361 [D loss: 0.000058, acc.: 100.00%] [G loss: 0.072960]
9362 [D loss: 0.000303, acc.: 100.00%] [G loss: 0.057480]
9363 [D loss: 0.000236, acc.: 100.00%] [G loss: 0.034186]
9364 [D loss: 0.000121, acc.: 100.00%] [G loss: 0.066656]
9365 [D loss: 0.000175, acc.: 100.00%] [G loss: 0.047534]
9366 [D loss: 0.000907, acc.: 100.00%] [G loss: 0.039079]
9367 [D loss: 0.000089, acc.: 100.00%] [G loss: 0.043576]
9368 [D loss: 0.000089, acc.: 100.00%] [G loss: 0.049230]
9369 [D loss: 0.000133, acc.: 100.00%] [G loss: 0.022921]
9370 [D loss: 0.000092, acc.: 100.00%] [G loss: 0.036405]
9371 [D loss: 0.000177, acc.: 100.00%] [G loss: 0.056319]
9372 [D loss: 0.000092, acc.: 100.00%] [G loss: 0.101631]
9373 [D loss: 0.000090, acc.: 100.00%] [G loss: 0.031667]
9374 [D loss: 0.000180, acc.: 100.00%] [G loss: 0.040461]
9375 [D loss: 0.000272, acc.: 100.00%] [G loss: 0.067624]
9376 [D loss: 

9501 [D loss: 0.000134, acc.: 100.00%] [G loss: 0.051669]
9502 [D loss: 0.000139, acc.: 100.00%] [G loss: 0.041852]
9503 [D loss: 0.000045, acc.: 100.00%] [G loss: 0.041182]
9504 [D loss: 0.000079, acc.: 100.00%] [G loss: 0.023660]
9505 [D loss: 0.000114, acc.: 100.00%] [G loss: 0.033074]
9506 [D loss: 0.000171, acc.: 100.00%] [G loss: 0.058605]
9507 [D loss: 0.000236, acc.: 100.00%] [G loss: 0.044618]
9508 [D loss: 0.000604, acc.: 100.00%] [G loss: 0.053681]
9509 [D loss: 0.000101, acc.: 100.00%] [G loss: 0.047485]
9510 [D loss: 0.000114, acc.: 100.00%] [G loss: 0.055089]
9511 [D loss: 0.000071, acc.: 100.00%] [G loss: 0.061266]
9512 [D loss: 0.000113, acc.: 100.00%] [G loss: 0.025431]
9513 [D loss: 0.000214, acc.: 100.00%] [G loss: 0.065942]
9514 [D loss: 0.000057, acc.: 100.00%] [G loss: 0.037154]
9515 [D loss: 0.000122, acc.: 100.00%] [G loss: 0.031481]
9516 [D loss: 0.000150, acc.: 100.00%] [G loss: 0.055464]
9517 [D loss: 0.001083, acc.: 100.00%] [G loss: 0.053730]
9518 [D loss: 

9645 [D loss: 0.000187, acc.: 100.00%] [G loss: 0.102855]
9646 [D loss: 0.000370, acc.: 100.00%] [G loss: 0.043855]
9647 [D loss: 0.000032, acc.: 100.00%] [G loss: 0.058632]
9648 [D loss: 0.000078, acc.: 100.00%] [G loss: 0.053189]
9649 [D loss: 0.000120, acc.: 100.00%] [G loss: 0.064281]
9650 [D loss: 0.000123, acc.: 100.00%] [G loss: 0.065590]
9651 [D loss: 0.000041, acc.: 100.00%] [G loss: 0.045575]
9652 [D loss: 0.001250, acc.: 100.00%] [G loss: 0.034268]
9653 [D loss: 0.000247, acc.: 100.00%] [G loss: 0.076510]
9654 [D loss: 0.000038, acc.: 100.00%] [G loss: 0.045433]
9655 [D loss: 0.000049, acc.: 100.00%] [G loss: 0.044725]
9656 [D loss: 0.000059, acc.: 100.00%] [G loss: 0.071931]
9657 [D loss: 0.000066, acc.: 100.00%] [G loss: 0.049723]
9658 [D loss: 0.000122, acc.: 100.00%] [G loss: 0.068989]
9659 [D loss: 0.000043, acc.: 100.00%] [G loss: 0.032984]
9660 [D loss: 0.000147, acc.: 100.00%] [G loss: 0.048608]
9661 [D loss: 0.000056, acc.: 100.00%] [G loss: 0.036787]
9662 [D loss: 

9787 [D loss: 0.000097, acc.: 100.00%] [G loss: 0.033188]
9788 [D loss: 0.000080, acc.: 100.00%] [G loss: 0.042113]
9789 [D loss: 0.000077, acc.: 100.00%] [G loss: 0.027752]
9790 [D loss: 0.000020, acc.: 100.00%] [G loss: 0.037032]
9791 [D loss: 0.000112, acc.: 100.00%] [G loss: 0.047847]
9792 [D loss: 0.000101, acc.: 100.00%] [G loss: 0.034679]
9793 [D loss: 0.000040, acc.: 100.00%] [G loss: 0.056462]
9794 [D loss: 0.000045, acc.: 100.00%] [G loss: 0.039694]
9795 [D loss: 0.000102, acc.: 100.00%] [G loss: 0.055664]
9796 [D loss: 0.000046, acc.: 100.00%] [G loss: 0.040958]
9797 [D loss: 0.000050, acc.: 100.00%] [G loss: 0.035560]
9798 [D loss: 0.000122, acc.: 100.00%] [G loss: 0.024979]
9799 [D loss: 0.000056, acc.: 100.00%] [G loss: 0.035385]
9800 [D loss: 0.000063, acc.: 100.00%] [G loss: 0.030373]
9801 [D loss: 0.000033, acc.: 100.00%] [G loss: 0.058568]
9802 [D loss: 0.000062, acc.: 100.00%] [G loss: 0.043106]
9803 [D loss: 0.000044, acc.: 100.00%] [G loss: 0.051422]
9804 [D loss: 

9932 [D loss: 0.000051, acc.: 100.00%] [G loss: 0.104150]
9933 [D loss: 0.000103, acc.: 100.00%] [G loss: 0.130034]
9934 [D loss: 0.000107, acc.: 100.00%] [G loss: 0.081732]
9935 [D loss: 0.000064, acc.: 100.00%] [G loss: 0.057516]
9936 [D loss: 0.000097, acc.: 100.00%] [G loss: 0.112094]
9937 [D loss: 0.000024, acc.: 100.00%] [G loss: 0.069600]
9938 [D loss: 0.000051, acc.: 100.00%] [G loss: 0.055596]
9939 [D loss: 0.000032, acc.: 100.00%] [G loss: 0.054866]
9940 [D loss: 0.000067, acc.: 100.00%] [G loss: 0.056157]
9941 [D loss: 0.000040, acc.: 100.00%] [G loss: 0.058589]
9942 [D loss: 0.000032, acc.: 100.00%] [G loss: 0.073802]
9943 [D loss: 0.000066, acc.: 100.00%] [G loss: 0.044011]
9944 [D loss: 0.000070, acc.: 100.00%] [G loss: 0.057240]
9945 [D loss: 0.000096, acc.: 100.00%] [G loss: 0.069754]
9946 [D loss: 0.000147, acc.: 100.00%] [G loss: 0.057645]
9947 [D loss: 0.000022, acc.: 100.00%] [G loss: 0.034375]
9948 [D loss: 0.000113, acc.: 100.00%] [G loss: 0.034978]
9949 [D loss: 

In [2]:
l

NameError: name 'l' is not defined